In [3]:
#get subset of MUSC records where journals equal those two chosen.

import xml.etree.ElementTree as ET
from xml.etree.ElementTree import Element
import pickle
import os
import pandas as pd
import numpy as np
import nltk.data

In [4]:
#on VM
os.chdir('/mnt/mypartition/Desktop2/pubmed_nlp_research/DeepLearningMovies_datasets/')
import KaggleWord2VecUtility

In [13]:
#on local:
import sys
sys.path.append('/home/sarahwie/Documents/pubmed-nlp-research/DeepLearningMovies_datasets/')
from KaggleWord2VecUtility import KaggleWord2VecUtility
DATADIR='/home/sarahwie/Documents/pubmed-nlp-research/DeepLearningMovies_datasets/'

In [17]:
#one-time run
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [5]:
# Load the punkt tokenizer
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

### Getting subset based on 2 journals (with abstracts and no Unicode encoding)

In [6]:
tree = ET.parse('/mnt/mypartition/Desktop2/testXMLwrite.xml')
root = tree.getroot()

In [7]:
#using this file for our example
xml_file = ET.parse('/home/sarahwie/Documents/zip/subset/zip/medline16n0189.xml')
root = xml_file.getroot()

This code subsets the records down to those with the titles we are looking for (as well as those with abstracts).

In [22]:
#Can delete I think for real run
#make new XML file
rootNew = Element('MedlineCitationSet')
children = []

In [23]:
#can delete I think for real run
#replace titles here with those we're using
titles = ['Physical review letters', 'Stroke; a journal of cerebral circulation']

i = 0
for record in root.findall('MedlineCitation'):
    title = record.find('Article').find('Journal').find('Title').text
    
    if title in titles:
        #for test run, also check if abstract
        #try:
        abstract = record.find('Article').find('Abstract').find('AbstractText').text
        if type(abstract) != str:
            i = i + 1
            print type(abstract)
        #append to subset file
        children.append(record)
        #except AttributeError:
            #dont include if no abstract
            #something random to hold place of indent:
            #x = 0

<type 'unicode'>
<type 'unicode'>
<type 'unicode'>
<type 'unicode'>
<type 'unicode'>
<type 'unicode'>
<type 'unicode'>
<type 'unicode'>
<type 'unicode'>
<type 'unicode'>
<type 'unicode'>
<type 'unicode'>
<type 'unicode'>
<type 'unicode'>
<type 'unicode'>
<type 'unicode'>
<type 'unicode'>
<type 'unicode'>
<type 'unicode'>
<type 'unicode'>
<type 'unicode'>
<type 'unicode'>
<type 'unicode'>
<type 'unicode'>
<type 'unicode'>
<type 'unicode'>
<type 'unicode'>
<type 'unicode'>
<type 'unicode'>
<type 'unicode'>
<type 'unicode'>
<type 'unicode'>
<type 'unicode'>
<type 'unicode'>
<type 'unicode'>
<type 'unicode'>
<type 'unicode'>
<type 'unicode'>
<type 'unicode'>
<type 'unicode'>
<type 'unicode'>
<type 'unicode'>
<type 'unicode'>
<type 'unicode'>
<type 'unicode'>
<type 'unicode'>
<type 'unicode'>
<type 'unicode'>
<type 'unicode'>
<type 'unicode'>
<type 'unicode'>
<type 'unicode'>
<type 'unicode'>
<type 'unicode'>
<type 'unicode'>
<type 'unicode'>
<type 'unicode'>
<type 'unicode'>
<type 'unicode

In [24]:
rootNew.extend(children)

#create tree and write to file
tree = ET.ElementTree(rootNew)
tree.write("subsetTwoJournals.xml")
        
pickle.dump(rootNew, open('rootTwoJournals.p', 'wb'))

In [25]:
print i

69


In [26]:
#Can delete I think for real run
#size of new subset
print len(rootNew.getchildren())
#verify that this is the correct number: yes

441


/home/nlp-vm/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:3: DeprecationWarning: This method will be removed in future versions.  Use 'list(elem)' or iteration over elem instead.
  app.launch_new_instance()


### Getting training and testing sets as pandas dataframes

In [27]:
#convert XML subset to pandas dataframe
def iter_docs(author):
    author_attr = author.attrib
    titles_pos = ['Physical review letters']
    titles_neg = ['Stroke; a journal of cerebral circulation']
    for record in author.findall('MedlineCitation'):
        doc_dict = author_attr.copy()
        #doc_dict.update(record.attrib)
        title = record.find('Article').find('Journal').find('Title').text
        doc_dict['journal'] = title
        abstract = record.find('Article').find('Abstract').find('AbstractText').text
        #make sure all abstracts are in string format to avoid later sentence parsing issues (69 in test run were unicode)
        if type(abstract) != str:
            abstract = abstract.encode('utf8')
        doc_dict['abstract'] = abstract
        #add sentiment label based on journal title
        if title in titles_neg:
            doc_dict['sentiment'] = 0
        elif title in titles_pos:
            doc_dict['sentiment'] = 1
        yield doc_dict
        
df = pd.DataFrame(list(iter_docs(rootNew)))
df = df[['sentiment', 'journal', 'abstract']]
print df

     sentiment                                    journal  \
0            0  Stroke; a journal of cerebral circulation   
1            0  Stroke; a journal of cerebral circulation   
2            0  Stroke; a journal of cerebral circulation   
3            0  Stroke; a journal of cerebral circulation   
4            0  Stroke; a journal of cerebral circulation   
5            0  Stroke; a journal of cerebral circulation   
6            0  Stroke; a journal of cerebral circulation   
7            0  Stroke; a journal of cerebral circulation   
8            0  Stroke; a journal of cerebral circulation   
9            0  Stroke; a journal of cerebral circulation   
10           0  Stroke; a journal of cerebral circulation   
11           0  Stroke; a journal of cerebral circulation   
12           0  Stroke; a journal of cerebral circulation   
13           0  Stroke; a journal of cerebral circulation   
14           0  Stroke; a journal of cerebral circulation   
15           0  Stroke; 

In [28]:
print df.loc[1][1]
#don't know why no quotation marks like when using pandas.to_csv in original w2v inversion
#but seems to work fine so ignore for now
df.loc[0:2]

Stroke; a journal of cerebral circulation


,sentiment,journal,abstract
0,0,Stroke; a journal of cerebral circulation,A standardized neurologic assessment scoring i...
1,0,Stroke; a journal of cerebral circulation,Proven effective therapy to prevent ischemic d...
2,0,Stroke; a journal of cerebral circulation,We describe a case of homozygous sickle cell a...


In [29]:
#split into train and test
#since so little data, consider using cross-validation?
fracTrain = 0.75
nSamples = df.shape[0]
order = np.random.permutation(nSamples) # come up with a random ordering
splitIndex = int(np.round(nSamples*fracTrain))
train1 = df.ix[order[:splitIndex],:]
test1 = df.ix[order[splitIndex:],:]

In [30]:
print train1.shape
print test1.shape

(331, 3)
(110, 3)


In [31]:
#print first record in test set (note has different index value, indexed instead by location (pos. 0))
print test1.iloc[[0]]

     sentiment                  journal  \
177          1  Physical review letters   

                                              abstract  
177  We report a measurement of the Lambda b0 lifet...  


### Parsing Sentences & Building Word2Vec Model

Gets a list of positive review indexes and negative review indexes. Takes those respective reviews and parses them into sentences and appends to a list of positive and negative sentences, respectively. Also does the same for the unlabelled training sentences.

In [34]:
# ****** Split the training set into clean sentences
#
sentences_pos = []  # Initialize an empty list of sentences
sentences_neg = []  # Initialize an empty list of sentences

#here change to include all journal name labels of positive and negative
inxs_pos = np.where(train1['sentiment'] == 1)[0].tolist()
inxs_neg = np.where(train1['sentiment'] == 0)[0].tolist()

print "Parsing sentences from training set"
for inx in inxs_pos:
    review = train1["abstract"].iloc[inx]
    sentences_pos += KaggleWord2VecUtility.review_to_sentences(review, tokenizer)

for inx in inxs_neg:
    review = train1["abstract"].iloc[inx]
    sentences_neg += KaggleWord2VecUtility.review_to_sentences(review, tokenizer)    

Parsing sentences from training set


/home/nlp-vm/anaconda2/lib/python2.7/site-packages/bs4/__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


In [35]:
print sentences_pos[0:2]

[[u'we', u'report', u'the', u'first', u'observation', u'of', u'diffractive', u'j', u'psi', u'mu', u'mu', u'production', u'in', u'pp', u'collisions', u'at', u'root', u'square', u's', u'tev'], [u'diffractive', u'events', u'are', u'identified', u'by', u'their', u'rapidity', u'gap', u'signature']]


Now do the same with all sentences to build the vocabulary

In [36]:
# ****** Split the labeled and unlabeled training sets into clean sentences
#
sentences = []  # Initialize an empty list of sentences

print "Parsing sentences from training set"
for review in train1["abstract"]:
    sentences += KaggleWord2VecUtility.review_to_sentences(review, tokenizer)

Parsing sentences from training set


Build Word2Vec model

In [37]:
from gensim.models import Word2Vec
import multiprocessing

## create a w2v learner 
basemodel = Word2Vec(
    workers=multiprocessing.cpu_count(), # use your cores
    iter=3, # iter = sweeps of SGD through the data; more is better
    hs=1, negative=0 # we only have scoring for the hierarchical softmax setup
    )
print basemodel

Word2Vec(vocab=0, size=100, alpha=0.025)


In [38]:
basemodel.build_vocab(sentences) 

Train Word2Vec model:

In [39]:
#model 0 corresponds with Stroke
#and model 1 with Physical Review Letters

from copy import deepcopy
models = [deepcopy(basemodel) for i in range(2)]
models[0].train(sentences_neg, total_examples=len(sentences_neg) )
models[1].train(sentences_pos, total_examples=len(sentences_pos) )

34731

### Applying inversion on the test set

In [41]:
"""
docprob takes two lists
* docs: a list of documents, each of which is a list of sentences
* models: the candidate word2vec models (each potential class)

it returns the array of class probabilities.  Everything is done in-memory.
"""

import pandas as pd # for quick summing within doc

def docprob(docs, mods):
    # score() takes a list [s] of sentences here; could also be a sentence generator
    sentlist = [s for d in docs for s in d]
    # the log likelihood of each sentence in this review under each w2v representation
    llhd = np.array( [ m.score(sentlist, len(sentlist)) for m in mods ] )
    # now exponentiate to get likelihoods, 
    lhd = np.exp(llhd - llhd.max(axis=0)) # subtract row max to avoid numeric overload
    # normalize across models (stars) to get sentence-star probabilities
    #all this transposing business does is make it so the total probability of a word 
    #   equals 1 between the 2 arrays (positive prob and negative).
    #and the pandas data frame just puts everything into rows/columns format for easy viz
    prob = pd.DataFrame( (lhd/lhd.sum(axis=0)).transpose() )
    # and finally average the sentence probabilities to get the review probability
    prob["doc"] = [i for i,d in enumerate(docs) for s in d]
    prob = prob.groupby("doc").mean()
    return prob

In [40]:
# read in the test set as a list of a list of words
docs = []
for review in test1["abstract"]:
    docs.append(KaggleWord2VecUtility.review_to_sentences(review, tokenizer))

In [42]:
print docs[0]

[[u'we', u'report', u'a', u'measurement', u'of', u'the', u'lambda', u'b', u'lifetime', u'in', u'the', u'exclusive', u'decay', u'lambda', u'b', u'j', u'psi', u'lambda', u'in', u'pp', u'collisions', u'at', u'square', u'root', u's', u'tev', u'using', u'an', u'integrated', u'luminosity', u'of', u'fb', u'of', u'data', u'collected', u'by', u'the', u'cdf', u'ii', u'detector', u'at', u'the', u'fermilab', u'tevatron'], [u'using', u'fully', u'reconstructed', u'decays', u'we', u'measure', u'tau', u'lambda', u'b', u'stat', u'syst', u'ps'], [u'this', u'is', u'the', u'single', u'most', u'precise', u'measurement', u'of', u'tau', u'lambda', u'b', u'and', u'is', u'sigma', u'higher', u'than', u'the', u'current', u'world', u'average']]


In [43]:
# get the probs (note we give docprob our test set plus the models)
probs = docprob(docs,models).astype(object)

In [44]:
probs

,0,1
doc,,
0,0.637194,0.362807
1,0.874565,0.125435
2,0.945536,0.0544643
3,0.945557,0.0544429
4,0.844883,0.155117
5,0.999294,0.000706446
6,0.947303,0.052697
7,0.942912,0.0570877
8,0.993642,0.00635769


In [45]:
predictions = np.ones((probs.shape[0]))

In [46]:
print np.shape(predictions)

(110,)


In [47]:
predictions[np.where(probs.iloc[:,0] > 0.5)] = 0 # The first column is the negative model

In [48]:
predictions

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,
        0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,
        0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,
        0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.])

In [49]:
print predictions.shape
print test1["sentiment"].shape

(110,)
(110,)


In [50]:
print np.size(np.where(predictions == test1["sentiment"]))*1./np.size(predictions)

0.581818181818


### Import KaggleWord2VecUtility since didn't work from file

In [33]:
import re
import nltk

import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
from nltk.corpus import stopwords


class KaggleWord2VecUtility(object):
    """KaggleWord2VecUtility is a utility class for processing raw HTML text into segments for further learning"""

    @staticmethod
    def review_to_wordlist( review, remove_stopwords=False ):
        # Function to convert a document to a sequence of words,
        # optionally removing stop words.  Returns a list of words.
        #
        # 1. Remove HTML
        review_text = BeautifulSoup(review).get_text()
        #
        # 2. Remove non-letters
        review_text = re.sub("[^a-zA-Z]"," ", review_text)
        #
        # 3. Convert words to lower case and split them
        words = review_text.lower().split()
        #
        # 4. Optionally remove stop words (false by default)
        if remove_stopwords:
            stops = set(stopwords.words("english"))
            words = [w for w in words if not w in stops]
        #
        # 5. Return a list of words
        return(words)

    # Define a function to split a review into parsed sentences
    @staticmethod
    def review_to_sentences( review, tokenizer, remove_stopwords=False ):
        # Function to split a review into parsed sentences. Returns a
        # list of sentences, where each sentence is a list of words
        #
        # 1. Use the NLTK tokenizer to split the paragraph into sentences
        raw_sentences = tokenizer.tokenize(review.decode('utf8').strip())
        #
        # 2. Loop over each sentence
        sentences = []
        for raw_sentence in raw_sentences:
            # If a sentence is empty, skip it
            if len(raw_sentence) > 0:
                # Otherwise, call review_to_wordlist to get a list of words
                sentences.append( KaggleWord2VecUtility.review_to_wordlist( raw_sentence, \
                  remove_stopwords ))
        #
        # Return the list of sentences (each sentence is a list of words,
        # so this returns a list of lists
        return sentences